# Thematic Color Analysis: Portraits, Landscapes, and Still Life

## Introduction

Artists don't use color randomly—they adapt their palettes based on **subject matter**. A portrait demands different colors than a seascape; a still life with flowers differs from one with fruit.

In this lesson, we'll computationally analyze how artists adjust their color choices based on what they're painting.

### What You'll Learn

- How to filter WikiArt by **genre** (portrait, landscape, still life, etc.)
- Statistical comparison of color usage across themes
- How individual artists vary their palette by subject
- Visualization techniques for thematic color analysis
- The relationship between subject matter and color temperature

### Key Concepts

- **Genre**: The category of subject matter (portrait, landscape, still life, etc.)
- **Thematic Palette**: The characteristic colors associated with a subject type
- **Color Adaptation**: How artists modify their palette for different subjects

Let's explore how theme shapes color!

## Setup

First, let's import our tools:

In [ ]:
from renoir import ArtistAnalyzer
from renoir.color import ColorExtractor, ColorAnalyzer, ColorVisualizer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

# Initialize analyzers
artist_analyzer = ArtistAnalyzer()
color_extractor = ColorExtractor()
color_analyzer = ColorAnalyzer()
visualizer = ColorVisualizer()

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

## Part 1: Understanding WikiArt Genres

WikiArt categorizes artworks by genre. Let's explore what genres are available and how they're distributed.

In [ ]:
# Load the dataset to explore genres
print("Loading WikiArt dataset...")
dataset = artist_analyzer._load_dataset()

# Get genre information
if hasattr(dataset, 'features') and 'genre' in dataset.features:
    genre_feature = dataset.features['genre']
    if hasattr(genre_feature, 'names'):
        genre_names = genre_feature.names
        print(f"\nFound {len(genre_names)} genres in WikiArt:\n")
        for i, genre in enumerate(genre_names):
            print(f"  {i}: {genre}")
else:
    print("Genre information not available in expected format")

### Key Genres for Our Analysis

We'll focus on three major genres that have distinct color characteristics:

1. **Portrait**: Focus on human subjects, often with skin tones and neutral backgrounds
2. **Landscape**: Natural scenes with greens, blues, earth tones
3. **Still Life**: Arranged objects, often with rich, saturated colors

Let's create a helper function to extract works by genre:

In [ ]:
def extract_works_by_genre(dataset, genre_name, limit=20):
    """
    Extract artworks of a specific genre from the WikiArt dataset.
    
    Args:
        dataset: The WikiArt dataset
        genre_name: Name of the genre to filter (e.g., 'portrait', 'landscape')
        limit: Maximum number of works to return
    
    Returns:
        List of artwork dictionaries
    """
    # Get genre names and find the target index
    genre_names = dataset.features['genre'].names
    
    # Find matching genre (case-insensitive partial match)
    target_idx = None
    for idx, name in enumerate(genre_names):
        if genre_name.lower() in name.lower():
            target_idx = idx
            print(f"Found genre: '{name}' (index {idx})")
            break
    
    if target_idx is None:
        print(f"Genre '{genre_name}' not found")
        return []
    
    # Filter dataset
    works = []
    for item in dataset:
        if item['genre'] == target_idx:
            works.append(item)
            if len(works) >= limit:
                break
    
    print(f"Extracted {len(works)} {genre_name} works")
    return works

# Test with portrait genre
print("Testing genre extraction...\n")
test_works = extract_works_by_genre(dataset, 'portrait', limit=5)
print(f"\nSample work: {test_works[0].get('title', 'Untitled') if test_works else 'None'}")

## Part 2: Extracting Color Palettes by Genre

Now let's extract and compare color palettes from different genres.

In [ ]:
# Define genres to analyze
genres_to_analyze = ['portrait', 'landscape', 'still-life']

# Extract colors for each genre
genre_colors = {}
genre_works = {}

for genre in genres_to_analyze:
    print(f"\n{'='*50}")
    print(f"Analyzing {genre.upper()}")
    print('='*50)
    
    # Get works for this genre
    works = extract_works_by_genre(dataset, genre, limit=15)
    genre_works[genre] = works
    
    if not works:
        continue
    
    # Extract colors from each work
    all_colors = []
    for work in works:
        try:
            palette = color_extractor.extract_dominant_colors(work['image'], n_colors=5)
            all_colors.extend(palette)
        except Exception as e:
            print(f"  Error processing work: {e}")
    
    genre_colors[genre] = all_colors
    print(f"  Total colors extracted: {len(all_colors)}")

print("\n" + "="*50)
print("EXTRACTION COMPLETE")
print("="*50)

### Visualize Genre Palettes

Let's create a visual comparison of the color palettes from each genre:

In [ ]:
# Create palette visualization for each genre
fig, axes = plt.subplots(len(genre_colors), 1, figsize=(16, 3 * len(genre_colors)))

if len(genre_colors) == 1:
    axes = [axes]

for ax, (genre, colors) in zip(axes, genre_colors.items()):
    if not colors:
        ax.text(0.5, 0.5, f'No colors for {genre}', ha='center', va='center')
        continue
    
    # Take a sample of colors to display
    sample_colors = colors[:30] if len(colors) > 30 else colors
    
    # Normalize colors for matplotlib
    colors_normalized = [(r/255, g/255, b/255) for r, g, b in sample_colors]
    
    # Create color swatches
    for i, color in enumerate(colors_normalized):
        ax.add_patch(plt.Rectangle((i, 0), 1, 1, facecolor=color, edgecolor='white', linewidth=0.5))
    
    ax.set_xlim(0, len(colors_normalized))
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(f"{genre.upper()} - Dominant Colors ({len(colors)} total)", 
                 fontsize=14, fontweight='bold', pad=10)

plt.suptitle('Color Palettes by Genre', fontsize=18, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Part 3: Statistical Analysis by Genre

Now let's compute and compare color statistics across genres.

In [ ]:
# Compute statistics for each genre
genre_stats = {}

for genre, colors in genre_colors.items():
    if not colors:
        continue
    
    # Basic statistics
    stats = color_analyzer.analyze_palette_statistics(colors)
    
    # Additional metrics
    temp_dist = color_analyzer.analyze_color_temperature_distribution(colors)
    diversity = color_analyzer.calculate_color_diversity(colors)
    saturation = color_analyzer.calculate_saturation_score(colors)
    brightness = color_analyzer.calculate_brightness_score(colors)
    
    genre_stats[genre] = {
        'mean_hue': stats['mean_hue'],
        'mean_saturation': stats['mean_saturation'],
        'mean_brightness': stats['mean_value'],
        'warm_percentage': temp_dist['warm_percentage'],
        'cool_percentage': temp_dist['cool_percentage'],
        'neutral_percentage': temp_dist['neutral_percentage'],
        'diversity': diversity,
        'dominant_temp': temp_dist['dominant_temperature']
    }

# Display as DataFrame
stats_df = pd.DataFrame(genre_stats).T
stats_df = stats_df.round(2)

print("\n" + "="*80)
print("COLOR STATISTICS BY GENRE")
print("="*80)
print(stats_df.to_string())

### Visualize Statistical Comparison

In [ ]:
# Create comparison visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

genres = list(genre_stats.keys())
colors_plot = ['#e74c3c', '#27ae60', '#3498db']

# Plot 1: Temperature Distribution
ax = axes[0, 0]
x = np.arange(len(genres))
width = 0.25

warm_vals = [genre_stats[g]['warm_percentage'] for g in genres]
cool_vals = [genre_stats[g]['cool_percentage'] for g in genres]
neutral_vals = [genre_stats[g]['neutral_percentage'] for g in genres]

ax.bar(x - width, warm_vals, width, label='Warm', color='#ff6b6b')
ax.bar(x, cool_vals, width, label='Cool', color='#4ecdc4')
ax.bar(x + width, neutral_vals, width, label='Neutral', color='#95a5a6')

ax.set_ylabel('Percentage (%)', fontweight='bold')
ax.set_title('Color Temperature by Genre', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([g.title() for g in genres])
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Plot 2: Saturation Comparison
ax = axes[0, 1]
sat_vals = [genre_stats[g]['mean_saturation'] for g in genres]
x_sat = np.arange(len(genres))
bars = ax.bar(x_sat, sat_vals, color=colors_plot)
ax.set_ylabel('Mean Saturation (%)', fontweight='bold')
ax.set_title('Average Saturation by Genre', fontsize=14, fontweight='bold')
ax.set_xticks(x_sat)
ax.set_xticklabels([g.title() for g in genres])
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bar, val in zip(bars, sat_vals):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{val:.1f}%', ha='center', fontweight='bold')

# Plot 3: Brightness Comparison
ax = axes[1, 0]
bright_vals = [genre_stats[g]['mean_brightness'] for g in genres]
x_bright = np.arange(len(genres))
bars = ax.bar(x_bright, bright_vals, color=colors_plot)
ax.set_ylabel('Mean Brightness (%)', fontweight='bold')
ax.set_title('Average Brightness by Genre', fontsize=14, fontweight='bold')
ax.set_xticks(x_bright)
ax.set_xticklabels([g.title() for g in genres])
ax.grid(axis='y', alpha=0.3)

for bar, val in zip(bars, bright_vals):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{val:.1f}%', ha='center', fontweight='bold')

# Plot 4: Color Diversity
ax = axes[1, 1]
div_vals = [genre_stats[g]['diversity'] for g in genres]
x_div = np.arange(len(genres))
bars = ax.bar(x_div, div_vals, color=colors_plot)
ax.set_ylabel('Diversity Score (0-1)', fontweight='bold')
ax.set_title('Color Diversity by Genre', fontsize=14, fontweight='bold')
ax.set_xticks(x_div)
ax.set_xticklabels([g.title() for g in genres])
ax.set_ylim(0, 1)
ax.grid(axis='y', alpha=0.3)

for bar, val in zip(bars, div_vals):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{val:.2f}', ha='center', fontweight='bold')

plt.suptitle('Color Characteristics by Genre', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Part 4: Hue Distribution Analysis

Let's visualize how hues are distributed across different genres using polar plots (color wheels).

In [ ]:
# Create hue distribution plots for each genre
fig, axes = plt.subplots(1, len(genre_colors), figsize=(6*len(genre_colors), 6),
                         subplot_kw=dict(projection='polar'))

if len(genre_colors) == 1:
    axes = [axes]

for ax, (genre, colors) in zip(axes, genre_colors.items()):
    if not colors:
        continue
    
    # Convert colors to HSV and extract hues
    hsv_colors = [color_analyzer.rgb_to_hsv(c) for c in colors]
    hues = [hsv[0] for hsv in hsv_colors]
    saturations = [hsv[1] for hsv in hsv_colors]
    
    # Convert hues to radians
    theta = np.radians(hues)
    
    # Use saturation as radius (more saturated = further out)
    r = [s / 100 for s in saturations]
    
    # Normalize colors for plotting
    color_norm = [(c[0]/255, c[1]/255, c[2]/255) for c in colors]
    
    ax.scatter(theta, r, c=color_norm, s=100, alpha=0.7, edgecolors='black', linewidth=0.5)
    
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_ylim(0, 1)
    ax.set_title(f'{genre.title()}\nHue Distribution', fontsize=14, fontweight='bold', pad=20)
    
    # Add hue labels
    ax.set_xticks(np.radians([0, 60, 120, 180, 240, 300]))
    ax.set_xticklabels(['Red', 'Yellow', 'Green', 'Cyan', 'Blue', 'Magenta'])

plt.suptitle('Hue Distribution by Genre\n(Distance from center = saturation)', 
             fontsize=16, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Portraits tend to cluster around warm skin tones (reds, oranges, yellows)")
print("- Landscapes often show blues and greens from sky and vegetation")
print("- Still life can be more varied depending on the objects depicted")

## Part 5: Single Artist Across Genres

Let's analyze how a single artist adapts their palette when painting different subjects.

In [ ]:
def get_artist_works_by_genre(dataset, artist_name, genre_name, limit=10):
    """
    Get works from a specific artist in a specific genre.
    """
    # Get artist and genre indices
    artist_names = dataset.features['artist'].names
    genre_names = dataset.features['genre'].names
    
    # Find artist index
    artist_idx = None
    for idx, name in enumerate(artist_names):
        if artist_name.lower() in name.lower():
            artist_idx = idx
            break
    
    # Find genre index
    genre_idx = None
    for idx, name in enumerate(genre_names):
        if genre_name.lower() in name.lower():
            genre_idx = idx
            break
    
    if artist_idx is None or genre_idx is None:
        return []
    
    # Filter for matching works
    works = []
    for item in dataset:
        if item['artist'] == artist_idx and item['genre'] == genre_idx:
            works.append(item)
            if len(works) >= limit:
                break
    
    return works

# Analyze Renoir across different genres
artist_name = 'pierre-auguste-renoir'
print(f"Analyzing {artist_name.title().replace('-', ' ')} across genres...\n")

artist_genre_colors = {}
genres_to_check = ['portrait', 'landscape', 'genre-painting', 'nude-painting-nu']

for genre in genres_to_check:
    works = get_artist_works_by_genre(dataset, artist_name, genre, limit=10)
    
    if works:
        colors = []
        for work in works:
            try:
                palette = color_extractor.extract_dominant_colors(work['image'], n_colors=5)
                colors.extend(palette)
            except:
                pass
        
        if colors:
            artist_genre_colors[genre] = colors
            print(f"  {genre}: {len(works)} works, {len(colors)} colors")

print(f"\nFound {len(artist_genre_colors)} genres with works")

In [ ]:
# Compare Renoir's palette across genres
if artist_genre_colors:
    # Compute statistics for each genre
    artist_stats = {}
    
    for genre, colors in artist_genre_colors.items():
        stats = color_analyzer.analyze_palette_statistics(colors)
        temp = color_analyzer.analyze_color_temperature_distribution(colors)
        
        artist_stats[genre] = {
            'Mean Hue': stats['mean_hue'],
            'Mean Saturation': stats['mean_saturation'],
            'Mean Brightness': stats['mean_value'],
            'Warm %': temp['warm_percentage'],
            'Cool %': temp['cool_percentage']
        }
    
    # Display
    artist_df = pd.DataFrame(artist_stats).T.round(1)
    print("\n" + "="*70)
    print(f"RENOIR'S COLOR ADAPTATION BY GENRE")
    print("="*70)
    print(artist_df.to_string())
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    genres = list(artist_stats.keys())
    
    # Temperature comparison
    ax = axes[0]
    warm = [artist_stats[g]['Warm %'] for g in genres]
    cool = [artist_stats[g]['Cool %'] for g in genres]
    
    x = np.arange(len(genres))
    width = 0.35
    
    ax.bar(x - width/2, warm, width, label='Warm', color='#ff6b6b')
    ax.bar(x + width/2, cool, width, label='Cool', color='#4ecdc4')
    ax.set_ylabel('Percentage (%)')
    ax.set_title("Renoir's Color Temperature by Subject", fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([g.replace('-', '\n').title() for g in genres], fontsize=10)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # Saturation/Brightness comparison
    ax = axes[1]
    sat = [artist_stats[g]['Mean Saturation'] for g in genres]
    bright = [artist_stats[g]['Mean Brightness'] for g in genres]
    
    ax.bar(x - width/2, sat, width, label='Saturation', color='#9b59b6')
    ax.bar(x + width/2, bright, width, label='Brightness', color='#f39c12')
    ax.set_ylabel('Percentage (%)')
    ax.set_title("Renoir's Saturation & Brightness by Subject", fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([g.replace('-', '\n').title() for g in genres], fontsize=10)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No genre data available for this artist")

## Part 6: Key Insights and Discussion

### What We've Learned

1. **Portraits** typically feature:
   - Warm colors (skin tones)
   - Moderate saturation
   - Focus on flesh tones and neutral backgrounds

2. **Landscapes** typically feature:
   - More cool colors (sky, water)
   - High color diversity
   - Mix of greens, blues, and earth tones

3. **Still Life** typically features:
   - High saturation (vibrant objects)
   - Varied color temperature
   - Depends heavily on depicted objects

### Questions for Reflection

1. **Historical Context**: How did available pigments affect genre palettes historically?

2. **Artist Style**: How do individual artists deviate from genre norms?

3. **Cultural Factors**: Do different cultures/periods show different genre-color associations?

4. **Your Observations**: What patterns did you notice in the visualizations?

## Exercise: Analyze Your Own Genre Comparison

Try comparing different genres or analyzing another artist's adaptation to subject matter.

In [ ]:
# YOUR CODE HERE
# Try different genres:
# - 'religious-painting'
# - 'mythological-painting'
# - 'cityscape'
# - 'marina' (seascapes)
# - 'flower-painting'
#
# Or try different artists:
# - 'claude-monet'
# - 'vincent-van-gogh'
# - 'rembrandt'
# - 'paul-cezanne'



---

## Conclusion

In this lesson, you learned:

- How to filter WikiArt by genre
- Statistical comparison of color usage across themes
- How individual artists adapt their palettes
- Visualization techniques for thematic analysis

**Key Takeaway**: Subject matter significantly influences color choices. Understanding these patterns helps us appreciate how artists make conscious decisions about their palettes based on what they're depicting.

### Next Steps

- Compare sub-genres (e.g., indoor vs outdoor portraits)
- Analyze seasonal themes in landscapes
- Study how historical periods affected genre palettes
- Build a genre classifier based on color features